In [1]:
import pandas as pd
import datetime
import numpy as np
import math

data_noise_hour = pd.read_csv("../Data/hourly_noisedata_2022.csv")
print(data_noise_hour.head())

print(len(data_noise_hour))

   month  day  hour                       description      lamax       laeq
0      1    1     0      MP 03: Naamsestraat 62 Taste  60.322528  57.126833
1      1    1     0    MP 05: Calvariekapel KU Leuven  53.230972  49.987639
2      1    1     0  MP 06: Parkstraat 2 La Filosovia  53.666056  50.752000
3      1    1     0            MP 07: Naamsestraat 81  50.056861  47.440222
4      1    1     1      MP 03: Naamsestraat 62 Taste  53.033583  50.853806
55469


In [2]:
data_weather_hour = pd.read_csv("../Data/hourly_weatherdata_2022.csv")
print(data_weather_hour.head())

print(len(data_weather_hour))

   Month  Day  Hour  LC_RAININ  LC_DAILYRAIN  LC_WINDDIR  LC_WINDSPEED   
0      1    1     0   0.000003       0.00036  -18.197324      0.389565  \
1      1    1     1   0.000007       0.00000  -16.227891      0.222602   
2      1    1     2   0.000009       0.00000  -13.710884      0.217194   
3      1    1     3   0.000000       0.00000  -16.401361      0.178248   
4      1    1     4   0.000000       0.00000  -10.268707      0.237670   

   LC_TEMP_QCL3  
0     13.100358  
1     12.669197  
2     12.520271  
3     12.386194  
4     12.080706  
8760


In [4]:
# Select the columns of interest from data_noise_hour
noise_columns = ["month", "day", "hour"]

# Select the columns of interest from data_weather_hour
weather_columns = ["Month", "Day", "Hour", "LC_RAININ", "LC_DAILYRAIN", "LC_WINDDIR", "LC_WINDSPEED","LC_TEMP_QCL3"]

# Merge the dataframes based on the common columns
data_model = data_noise_hour.merge(data_weather_hour[weather_columns], left_on=noise_columns, right_on=weather_columns[:-5], how='left')

#print(data_model.head(20))

# Remove the columns "Month", "Day", and "Hour" from data_model
data_model = data_model.drop(["Month", "Day", "Hour"], axis=1)

print(data_model.head(20))

    month  day  hour                       description      lamax       laeq   
0       1    1     0      MP 03: Naamsestraat 62 Taste  60.322528  57.126833  \
1       1    1     0    MP 05: Calvariekapel KU Leuven  53.230972  49.987639   
2       1    1     0  MP 06: Parkstraat 2 La Filosovia  53.666056  50.752000   
3       1    1     0            MP 07: Naamsestraat 81  50.056861  47.440222   
4       1    1     1      MP 03: Naamsestraat 62 Taste  53.033583  50.853806   
5       1    1     1    MP 05: Calvariekapel KU Leuven  53.599639  50.578806   
6       1    1     1  MP 06: Parkstraat 2 La Filosovia  59.880694  56.942167   
7       1    1     1            MP 07: Naamsestraat 81  50.097278  47.878333   
8       1    1     2      MP 03: Naamsestraat 62 Taste  52.173702  50.049903   
9       1    1     2    MP 05: Calvariekapel KU Leuven  51.078083  47.974361   
10      1    1     2  MP 06: Parkstraat 2 La Filosovia  52.107194  48.968806   
11      1    1     2            MP 07: N

In [58]:
# Remove the columns "Month", "Day", and "Hour" from data_model
#data_model = data_model.drop(["LC_RAININ"], axis=1)

#wind direction could have an impact as it "carries" the noise, could bring the noise to another place
#e.g. noise from the center is carried by the wind to the Naamse straat

In [ ]:
#transform the variables month day and hour to cyclic variables
#so that the model knows that hour 0 is close to hour 23, and month January is close to month December

#information about the week is missing



In [5]:
#add weekday information: feature from 1 to 7 indicating the day of the week
from dateutil.relativedelta import relativedelta

# Define the start date
start_date = datetime.datetime(2022, 1, 1)  # Assuming the year is 2022

# Add the "weekday" column
data_model['weekday'] = data_model.apply(lambda row: (start_date + relativedelta(days=row['day']-1, months=row['month']-1)).weekday() + 1, axis=1)

# Map weekday numbers to weekday names
weekday_names = np.array(range(1,8))
data_model['weekday'] = data_model['weekday'].map(lambda x: weekday_names[x-1])

data_model.head()



,month,day,hour,description,lamax,laeq,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,LC_TEMP_QCL3,weekday
0,1,1,0,MP 03: Naamsestraat 62 Taste,60.322528,57.126833,0.000003,0.00036,-18.197324,0.389565,13.100358,6
1,1,1,0,MP 05: Calvariekapel KU Leuven,53.230972,49.987639,0.000003,0.00036,-18.197324,0.389565,13.100358,6
2,1,1,0,MP 06: Parkstraat 2 La Filosovia,53.666056,50.752000,0.000003,0.00036,-18.197324,0.389565,13.100358,6
3,1,1,0,MP 07: Naamsestraat 81,50.056861,47.440222,0.000003,0.00036,-18.197324,0.389565,13.100358,6
4,1,1,1,MP 03: Naamsestraat 62 Taste,53.033583,50.853806,0.000007,0.00000,-16.227891,0.222602,12.669197,6


In [6]:
#third of March: Tuesday
#print(data_model.loc[6000])

print("other day")
#17th of May: Tuesday
#print(data_model.loc[18000])

print("one of the last days, see if complete")
#first of December: Thursday
print(data_model.loc[50000])

#control is ok

other day
one of the last days, see if complete
month                               12
day                                  1
hour                                 0
description     MP 07: Naamsestraat 81
lamax                        50.872444
laeq                         48.564472
LC_RAININ                     0.000009
LC_DAILYRAIN                  0.001454
LC_WINDDIR                   -0.984568
LC_WINDSPEED                  0.031636
LC_TEMP_QCL3                  4.999267
weekday                              4
Name: 50000, dtype: object


In [7]:
#plot the locations on a straight line

gps_data = {
    'description': ['MP 01: Naamsestraat 35  Maxim', 'MP 02: Naamsestraat 57 Xior', 'MP 03: Naamsestraat 62 Taste', 'MP 04: His & Hears', 'MP 05: Calvariekapel KU Leuven', 'MP 06: Parkstraat 2 La Filosovia', 'MP 07: Naamsestraat 81', 'MP08bis - Vrijthof'],
    'lat': [50.877121, 50.87650, 50.87590, 50.875237, 50.87452, 50.874078, 50.873808, 50.87873],
    'lon': [4.700708, 4.700632, 4.700262, 4.700071, 4.69985, 4.70005, 4.700007, 4.70115]
}

In [9]:
#transform to one line
# Extract coordinates and location names
latitudes = gps_data['lat']
longitudes = gps_data['lon']
locations = gps_data['description']

# Initialize variables to store the indices of the extreme locations
max_dist_idx_1 = 0
max_dist_idx_2 = 0
max_distance = 0

# Calculate distances and find the pair with maximum distance
for i in range(len(latitudes) - 1):
    for j in range(i + 1, len(latitudes)):
        # Calculate Euclidean distance
        distance = math.sqrt((latitudes[i] - latitudes[j]) ** 2 + (longitudes[i] - longitudes[j]) ** 2)

        # Update max_distance and indices if a larger distance is found
        if distance > max_distance:
            max_distance = distance
            max_dist_idx_1 = i
            max_dist_idx_2 = j

# Get the coordinates and names of the two most extreme locations
extreme_location_1 = (latitudes[max_dist_idx_1], longitudes[max_dist_idx_1], locations[max_dist_idx_1])
extreme_location_2 = (latitudes[max_dist_idx_2], longitudes[max_dist_idx_2], locations[max_dist_idx_2])

# Print the results
print("Extreme Location 1:", extreme_location_1)
print("Extreme Location 2:", extreme_location_2)

Extreme Location 1: (50.873808, 4.700007, 'MP 07: Naamsestraat 81')
Extreme Location 2: (50.87873, 4.70115, 'MP08bis - Vrijthof')


In [10]:
# Convert the dictionary to a DataFrame
gps_df = pd.DataFrame(gps_data)

print(gps_df)




                        description        lat       lon
0     MP 01: Naamsestraat 35  Maxim  50.877121  4.700708
1       MP 02: Naamsestraat 57 Xior  50.876500  4.700632
2      MP 03: Naamsestraat 62 Taste  50.875900  4.700262
3                MP 04: His & Hears  50.875237  4.700071
4    MP 05: Calvariekapel KU Leuven  50.874520  4.699850
5  MP 06: Parkstraat 2 La Filosovia  50.874078  4.700050
6            MP 07: Naamsestraat 81  50.873808  4.700007
7                MP08bis - Vrijthof  50.878730  4.701150


In [11]:
def project_point_onto_line(point, line_start, line_end):
    line_direction = np.array(line_end) - np.array(line_start)
    line_length = np.linalg.norm(line_direction)
    line_direction_normalized = line_direction / line_length
    
    point_vector = np.array(point) - np.array(line_start)
    projection_length = np.dot(point_vector, line_direction_normalized)
    projection = np.array(line_start) + projection_length * line_direction_normalized
    
    # Calculate the coordinate on the line between 0 and 1
    coordinate = np.linalg.norm(projection - np.array(line_start)) / line_length
    
    return coordinate

#basic tests return the right values
a = [0,0]
b=[1,1]
c=[0.5,0.5]

c_projected = project_point_onto_line(c,a,b)
print(c_projected)


a = [0,1]
b=[5,1]
c=[4,2]

c_projected = project_point_onto_line(c,a,b)
print(c_projected)

0.49999999999999983
0.8


In [12]:
# Define the starting and end points
extreme_location_1 = (latitudes[max_dist_idx_1], longitudes[max_dist_idx_1])
extreme_location_2 = (latitudes[max_dist_idx_2], longitudes[max_dist_idx_2])

# Apply the projection function to the DataFrame
gps_df['coordinate'] = gps_df.apply(lambda row: project_point_onto_line(
    (row['lat'], row['lon']), extreme_location_1, extreme_location_2), axis=1)

print(gps_df)

                        description        lat       lon  coordinate
0     MP 01: Naamsestraat 35  Maxim  50.877121  4.700708    0.670040
1       MP 02: Naamsestraat 57 Xior  50.876500  4.700632    0.546926
2      MP 03: Naamsestraat 62 Taste  50.875900  4.700262    0.414698
3                MP 04: His & Hears  50.875237  4.700071    0.278339
4    MP 05: Calvariekapel KU Leuven  50.874520  4.699850    0.130227
5  MP 06: Parkstraat 2 La Filosovia  50.874078  4.700050    0.053974
6            MP 07: Naamsestraat 81  50.873808  4.700007    0.000000
7                MP08bis - Vrijthof  50.878730  4.701150    1.000000


In [13]:
# Create a dictionary mapping descriptions to coordinates
description_to_coordinate = dict(zip(gps_df['description'], gps_df['coordinate']))

# Add the 'coordinate' column to data_model based on 'description'
data_model['coordinate'] = data_model['description'].map(description_to_coordinate)

# Print the updated data_model DataFrame
print(data_model.head())


   month  day  hour                       description      lamax       laeq   
0      1    1     0      MP 03: Naamsestraat 62 Taste  60.322528  57.126833  \
1      1    1     0    MP 05: Calvariekapel KU Leuven  53.230972  49.987639   
2      1    1     0  MP 06: Parkstraat 2 La Filosovia  53.666056  50.752000   
3      1    1     0            MP 07: Naamsestraat 81  50.056861  47.440222   
4      1    1     1      MP 03: Naamsestraat 62 Taste  53.033583  50.853806   

   LC_RAININ  LC_DAILYRAIN  LC_WINDDIR  LC_WINDSPEED  LC_TEMP_QCL3  weekday   
0   0.000003       0.00036  -18.197324      0.389565     13.100358        6  \
1   0.000003       0.00036  -18.197324      0.389565     13.100358        6   
2   0.000003       0.00036  -18.197324      0.389565     13.100358        6   
3   0.000003       0.00036  -18.197324      0.389565     13.100358        6   
4   0.000007       0.00000  -16.227891      0.222602     12.669197        6   

   coordinate  
0    0.414698  
1    0.130227  
2 

In [14]:
#for month feature
number_months = 12

angle_month = 360/number_months

data_model["sin_month"] = np.sin(np.deg2rad(data_model["month"] * angle_month))

data_model["cos_month"] = np.cos(np.deg2rad(data_model["month"] * angle_month))

#test correclty done
#print(data_model.head())
#print(data_model.tail())
print(data_model.loc[10000])
print(data_model.loc[30000])

month                                      3
day                                       31
hour                                       0
description     MP 03: Naamsestraat 62 Taste
lamax                              57.588302
laeq                               54.966852
LC_RAININ                           0.000009
LC_DAILYRAIN                        0.000031
LC_WINDDIR                           1.27381
LC_WINDSPEED                        0.051854
LC_TEMP_QCL3                        6.948204
weekday                                    4
coordinate                          0.414698
sin_month                                1.0
cos_month                                0.0
Name: 10000, dtype: object
month                                      7
day                                       28
hour                                       4
description     MP 03: Naamsestraat 62 Taste
lamax                              46.571167
laeq                                 45.1965
LC_RAININ                   

In [15]:
#for day feature

number_of_day_per_month = [31,28,31,30,31,30,31,31,30,31,30,31]

# Calculate the angle_day based on the number of days in each month
data_model["angle_day"] = 360 / np.array(number_of_day_per_month)[data_model["month"] - 1]

# Calculate sin_day and cos_day using the dynamic angle_day
data_model["sin_day"] = np.sin(np.deg2rad(data_model["day"] * data_model["angle_day"]))
data_model["cos_day"] = np.cos(np.deg2rad(data_model["day"] * data_model["angle_day"]))

# Drop the angle_day column
data_model = data_model.drop("angle_day", axis=1)

#test correclty done
print(data_model.head())
print(data_model.tail())
print(data_model.loc[10000])
print(data_model.loc[30000])

   month  day  hour                       description      lamax       laeq   
0      1    1     0      MP 03: Naamsestraat 62 Taste  60.322528  57.126833  \
1      1    1     0    MP 05: Calvariekapel KU Leuven  53.230972  49.987639   
2      1    1     0  MP 06: Parkstraat 2 La Filosovia  53.666056  50.752000   
3      1    1     0            MP 07: Naamsestraat 81  50.056861  47.440222   
4      1    1     1      MP 03: Naamsestraat 62 Taste  53.033583  50.853806   

   LC_RAININ  LC_DAILYRAIN  LC_WINDDIR  LC_WINDSPEED  LC_TEMP_QCL3  weekday   
0   0.000003       0.00036  -18.197324      0.389565     13.100358        6  \
1   0.000003       0.00036  -18.197324      0.389565     13.100358        6   
2   0.000003       0.00036  -18.197324      0.389565     13.100358        6   
3   0.000003       0.00036  -18.197324      0.389565     13.100358        6   
4   0.000007       0.00000  -16.227891      0.222602     12.669197        6   

   coordinate  sin_month  cos_month   sin_day  cos

In [16]:
#for hour feature

number_hours = 24

angle_hour = 360/number_hours

data_model["sin_hour"] = np.sin(np.deg2rad(data_model["hour"] * angle_hour))

data_model["cos_hour"] = np.cos(np.deg2rad(data_model["hour"] * angle_hour))

#test correclty done
#print(data_model.head())
#print(data_model.tail())
print(data_model.loc[10000])
print(data_model.loc[30000])

month                                      3
day                                       31
hour                                       0
description     MP 03: Naamsestraat 62 Taste
lamax                              57.588302
laeq                               54.966852
LC_RAININ                           0.000009
LC_DAILYRAIN                        0.000031
LC_WINDDIR                           1.27381
LC_WINDSPEED                        0.051854
LC_TEMP_QCL3                        6.948204
weekday                                    4
coordinate                          0.414698
sin_month                                1.0
cos_month                                0.0
sin_day                                 -0.0
cos_day                                  1.0
sin_hour                                 0.0
cos_hour                                 1.0
Name: 10000, dtype: object
month                                      7
day                                       28
hour                        

In [17]:
#for weekday feature

number_weekdays = 7

angle_weekday = 360/number_weekdays

data_model["sin_weekday"] = np.sin(np.deg2rad(data_model["weekday"] * angle_weekday))

data_model["cos_weekday"] = np.cos(np.deg2rad(data_model["weekday"] * angle_weekday))

#test correclty done
#print(data_model.head())
#print(data_model.tail())
print(data_model.loc[10000])
print(data_model.loc[30000])

month                                      3
day                                       31
hour                                       0
description     MP 03: Naamsestraat 62 Taste
lamax                              57.588302
laeq                               54.966852
LC_RAININ                           0.000009
LC_DAILYRAIN                        0.000031
LC_WINDDIR                           1.27381
LC_WINDSPEED                        0.051854
LC_TEMP_QCL3                        6.948204
weekday                                    4
coordinate                          0.414698
sin_month                                1.0
cos_month                                0.0
sin_day                                 -0.0
cos_day                                  1.0
sin_hour                                 0.0
cos_hour                                 1.0
sin_weekday                        -0.433884
cos_weekday                        -0.900969
Name: 10000, dtype: object
month                       

In [18]:
data_model.to_csv('data_model.csv', index=False)